In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(stringr)

lapply(c("dplyr","Seurat","HGNChelper","openxlsx"), library, character.only = T)

load('../data/reference.RData')

sample_meta<-'AD_HS_00001	Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease	20	H.Sapiens	AD	Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex	Braak:0;0;0;2;2;2;2;6;6;6;0;0;0;2;2;2;2;6;6;6	ExcitatoryNeuron;InhibitoryNeuron;Oligodendrocyte;OPC;Astrocyte;Microglia;Endothelia;Pericyte	GSE147528	Single-nucleus RNA-seq	10x Genomics	Male	60;50;71;72;87;91;77;82;72;82;60;50;71;72;87;91;77;82;72;82	SRR11422700;SRR11422701;SRR11422702;SRR11422703;SRR11422704;SRR11422705;SRR11422706;SRR11422707;SRR11422708;SRR11422709;SRR11422710;SRR11422711;SRR11422712;SRR11422713;SRR11422714;SRR11422715;SRR11422716;SRR11422717;SRR11422718;SRR11422719'
split_meta<-unlist(strsplit(sample_meta, '\t'))
samples<-unlist(strsplit(split_meta[14],';'))
disease<-rep(split_meta[5],length(samples))
stage<-unlist(strsplit(unlist(strsplit(split_meta[7],':'))[2],';'))
gender<-rep(split_meta[12],length(samples))
age<-unlist(strsplit(split_meta[13],';'))

sample_meta<-data.frame(samples=samples,disease=disease,stage=stage,gender=gender,age=age)
sample_meta[sample_meta$stage=='0','disease']<-'Control'
rownames(sample_meta)<-sample_meta$samples

# args <- commandArgs(trailingOnly=TRUE)
args<-c('../data/3/scte','../data/3/cell_umap.txt','SRR11422700','/home/wdeng3/workspace/software/sc-type','../../universal_data/rmsk/rmsk_GRCh38.txt')
data_path<-args[1]
out_path<-args[2]
sample_tag<-args[3]
sctype_path<-args[4]
rmsk_path<-args[5]
rmsk<-read.table(rmsk_path,sep='\t',header=F)
to_remove<-unlist(rmsk[!(rmsk$V12 %in% c('LINE','SINE','LTR')),'V11'])
to_remove<-str_replace_all(to_remove,'-','\\.')

reps<-unlist(rmsk[rmsk$V12 %in% c('LINE','SINE','LTR'),'V11'])
reps<-str_replace_all(reps,'-','\\.')

create_seurat<-function(sample_tag){
    ## load data
    scte.data <- t(read.csv(paste0(data_path,'/',sample_tag,'/',sample_tag,'.csv'),check.names=F, row.names = 1))
    # remove repeats from gene expression matrix

    scte.gene<-scte.data[!(row.names(scte.data) %in% to_remove),]
    # normalize and scale data
    scte <- CreateSeuratObject(counts = scte.gene, project = sample_tag, min.cells = 3, min.features = 200)
    # normalize data
    scte[["percent.mt"]] <- PercentageFeatureSet(scte, pattern = "^MT.|^MT-")
    scte <- subset(scte, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
    scte <- SCTransform(scte, vars.to.regress = "percent.mt", verbose = FALSE)

    cell_meta<-scte@meta.data
    cell_meta$disease<-sample_meta[cell_meta$orig.ident,'disease']

    cell_meta$stage<-sample_meta[cell_meta$orig.ident,'stage']
    cell_meta$gender<-sample_meta[cell_meta$orig.ident,'gender']
    cell_meta$age<-sample_meta[cell_meta$orig.ident,'age']
    scte@meta.data<-cell_meta
    scte
}

scte<-create_seurat(sample_tag)

scte <- FindVariableFeatures(scte, selection.method = "vst", nfeatures = 2000)
# scale and run PCA
scte <- ScaleData(scte, features = rownames(scte))
scte <- RunPCA(scte, features = VariableFeatures(object = scte))

sdv<-Stdev(scte,reduction='pca')
sdv<-sdv[sdv>2]
npca<-length(sdv)

scte <- FindNeighbors(scte, dims = 1:npca)
scte <- FindClusters(scte, resolution = 0.8)
scte <- RunUMAP(scte, dims = 1:npca)


scte.anchors <- FindTransferAnchors(reference = reference, query = scte,
    dims = 1:30, reference.reduction = "pca")
scte.query <- MapQuery(anchorset = scte.anchors, reference = reference, query = scte,
    refdata = list(celltype = "broad.cell.type"), reference.reduction = "pca", reduction.model = "umap")
cell_meta<-scte.query@meta.data

cell_meta$UMAP_1<-scte.query@reductions[['umap']]@cell.embeddings[,1]
cell_meta$UMAP_2<-scte.query@reductions[['umap']]@cell.embeddings[,2]
scte.query@meta.data<-cell_meta
write.table(cell_meta,out_path,sep='\t',quote=F,row.names=T)

## get gene expression from scte_query
scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)

In [14]:


import sys
import math
import tqdm
import pandas as pd
import os
import json
from collections import defaultdict

sys.argv=['th','../data/all_datasets','../www/mysql','overwrite','../../universal_data/ref/GRCh38/gencode.v43.basic.annotation.gtf','../../universal_data/rmsk/rmsk_GRCh38.txt']
inpath=sys.argv[1]
out_path=sys.argv[2]
mode='append' if sys.argv[3]=='append' else 'overwrite'
gtf=sys.argv[4]
rmsk_f=sys.argv[5]

if mode=='overwrite':
    genes=[]
    for line in open(gtf):
        if not line.startswith('#'):
            info=line.strip().split('\t')
            if info[2]=='gene' and ('lncRNA' in info[-1] or 'protein_coding' in info[-1]):
                split_='gene_name "' if 'gene_name' in info[-1] else 'gene_id "'
                genes.append(info[-1].split(split_)[1].split('"')[0])            

    rmsk=pd.read_csv(rmsk_f,sep='\t',header=None)
    rmsk=rmsk[rmsk[11].isin(['LINE','SINE','LTR'])]
    genes.extend(rmsk[10].unique())
    genes=list(set(genes))

    exp_tables=[open(f'{out_path}/cell_exp_{i}.sql','w') for i in range(math.ceil((len(genes))/1000.0))]
    for i in range(len(exp_tables)):
        exp_tables[i].write('''CREATE DATABASE IF NOT EXISTS scARE;
                                USE scARE;
                                DROP TABLE IF EXISTS CELL_EXP_{i}; 
                                CREATE TABLE CELL_EXP_{i} (
                                scARE_ID varchar(255) NOT NULL,
                                CELL varchar(255) NOT NULL,\n'''.format(i=i))
        
    gene_dict=open(f'{out_path}/gene_dict.sql','w') 
    gene_dict.write('''\
            CREATE DATABASE IF NOT EXISTS scARE;
            USE scARE;
            DROP TABLE IF EXISTS GENE_DICT;
            CREATE TABLE GENE_DICT (
                GENE varchar(255) NOT NULL,
                TABLE_ID INT NOT NULL);\n''')

    table_genes=defaultdict(list)
    for i in range(len(genes)):
        index=math.floor(i/1000.0)
        exp_tables[index].write(f'`{genes[i]}` FLOAT DEFAULT 0,\n')
        table_genes[index].append(genes[i])
        gene_dict.write(f'INSERT INTO GENE_DICT values("{genes[i]}","{index}");\n')
    for i in range(len(exp_tables)):
        exp_tables[i].write('UMAP_1 FLOAT NOT NULL,\n')
        exp_tables[i].write('UMAP_2 FLOAT NOT NULL);\n')
        exp_tables[i].write('set autocommit=0;\n')
        exp_tables[i].flush()
    json.dump(table_genes,open(f'{out_path}/table_genes.json','w'))
else:
    table_genes=json.load(open(f'{out_path}/table_genes.json','r'))
    exp_tables=[open(f'{out_path}/cell_exp_{i}.sql','a') for i in range(len(table_genes))]


def one_table(i):
    expressed_i=[x for x in cell_exp.columns if x in table_genes[i]]+['UMAP_1','UMAP_2']
    tmp_exp=pd.DataFrame(columns=(['scARE_ID','CELL']+expressed_i),index=cell_exp.index)
    tmp_exp['scARE_ID']=['"%s"'%cell_dataset[x] for x in cell_exp.index]
    tmp_exp['CELL']=['"%s"'%x for x in cell_exp.index]
    tmp_exp.loc[:,expressed_i]=cell_exp.loc[:,expressed_i]
    all_values=tmp_exp.agg(lambda x:  ','.join(x.astype(str)), axis=1).values
    expressed_i_c=','.join(expressed_i)
    template=f'''INSERT INTO CELL_EXP_{i} (scARE_ID,CELL,{expressed_i_c}) values '''
    count=0
    values_list=[]
    to_write=''
    for x in all_values:
        if count==2000:
            to_write+=template+','.join(values_list)+';\n'
            count=0
            values_list=[]
        values_list.append('('+x+')')
        count+=1
    if len(values_list)>0:
        to_write+=template+','.join(values_list)+';\n'
    # to_write='\n'.join([template.format(i=i,expressed_i_c=expressed_i_c,values=x) for x in all_values])+'\n'
    exp_tables[i].write(to_write)

from multiprocessing import Pool
for file_ in os.listdir(inpath):
    if file_.endswith('.cell_exp.txt'):
        cell_exp=pd.read_csv(os.path.join(inpath,file_),sep='\t',index_col=0)
        cell_dataset=pd.read_csv(os.path.join(inpath,file_.replace('cell_exp','cell_umap')),sep='\t',index_col=0)['dataset'].to_dict()
        pool=Pool(40)
        pool.map(one_table,range(len(exp_tables)))
        pool.close()
        pool.join()

[x.write('commit;\n') for x in exp_tables]
[x.flush() for x in exp_tables]
[x.close() for x in exp_tables]

/tmp/ipykernel_9533/1952009851.py:25: DtypeWarning: Columns (0,1,2,3,4,6,7,8,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  rmsk=pd.read_csv(rmsk_f,sep='\t',header=None)


In [ ]:
# ## assign cell type according to ScType tuturial

## load sc type and database
# source(paste0(sctype_path,"/R/gene_sets_prepare.R"))
# source(paste0(sctype_path,"/R/sctype_score_.R"))
# gs_list = gene_sets_prepare(paste0(sctype_path,'/ScTypeDB_full.xlsx'), "Brain" )
# # get cell-type by cell matrix
# es.max = sctype_score(scRNAseqData = scte[["RNA"]]@scale.data, scaled = TRUE, 
#                       gs = gs_list$gs_positive, gs2 = gs_list$gs_negative) 

# # merge by cluster
# cL_resutls = do.call("rbind", lapply(unique(scte@meta.data$seurat_clusters), function(cl){
#     es.max.cl = sort(rowSums(es.max[ ,rownames(scte@meta.data[scte@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
#     head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(scte@meta.data$seurat_clusters==cl)), 10)
# }))
# sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# # set low-confident (low ScType score) clusters to "unknown"
# sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"


# scte@meta.data$customclassif = ""
# for(j in unique(sctype_scores$cluster)){
#   cl_type = sctype_scores[sctype_scores$cluster==j,]; 
#   scte@meta.data$customclassif[scte@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
# }

# ## save cell type
# cell_types<-scte[['customclassif']]
